## Rule Based Label _ 10Hz

##### Warning! Check Acc/Gyro direction before run

In [38]:
import pandas as pd
import numpy as np

## DATA LOAD

file_path = '../Data/'
file_name = 'data_10Hz.csv'
df = pd.read_csv(file_path+file_name)

## 급가속
---
### 조건
1. Vehicle Speed more than __*10km/h*__ per second
2. Accelerate over __*6km/h*__ within __*1 sec*__.
3. Engine RPM exceed __*2500*__
4. Accelerator pedal position exceed __*0%*__
5. Longitudinal Acceleration exceed __*1.8m/s2*__ (2번 조건과 충돌됨, C2 : 1.67m/s2) _ 삭제


In [40]:
def get_stepgas(df, thres_speed = 10, thres_dspeed = 6, thres_long_acc = 1.67, thres_rpm = 2500, thres_acc_pdl_pos = 0, speed_in_ms = True):
    
    # Convert Speed in kph unit
    if speed_in_ms:
        df.Speed = df.Speed * 3.6;
        
    # Speed Mask
    mask_c1 = df.Speed > thres_speed
    
    # Speed Difference Mask
    dspeed = np.roll(df.Speed.values,-10) - df.Speed.values
    mask_c2 = (dspeed >= thres_dspeed)
    
    # RPM Mask
    mask_c3 = df.N > thres_rpm
    
    # Pedal Mask
    mask_c4 = df.PV_AV_CAN > thres_acc_pdl_pos
    
    # Longitudinal Acc Mask
    mask_c5 = df.AccelX > thres_long_acc

    # index_list = df.index[mask_c1 & mask_c2 & mask_c3 & mask_c4 & mask_c5]
    index_list = df.index[mask_c1 & mask_c2 & mask_c3 & mask_c4]
    
    return index_list

In [41]:
get_stepgas(df)

Int64Index([ 8848, 24230, 24231, 24232, 24233, 24234, 24235, 24236, 24238,
            24239, 24240, 24241, 24242, 24243, 43347, 43348, 43349, 43350,
            43351, 43352, 43353, 43354, 43355, 43356, 43357, 43358, 43359,
            43360, 43361, 43362, 43363, 43364, 43365, 43366, 43367, 43368,
            43369, 43370, 43371],
           dtype='int64')

## 급출발*
---
### 조건
1. Vehicle Speed under than __*10km/h*__ at initial point. [data.Speed]
2. Accelerate over __*8km/h*__ per second [data.Speed 변화량]
3. Engine RPM exceed __*2500*__ [data.N]
4. Accelerator pedal position exceed __*0%*__ [data.PV_AV_CAN]
5. Acceleration over __*1.8m/s2*__ (2번 조건과 충돌됨, C2 : 2.22m/s2) [data.AccelX]  _ 삭제

In [42]:
def get_suddendepature(df, thres_speed = 10, thres_dspeed = 8, thres_long_acc = 1.8, thres_rpm = 2500, thres_acc_pdl_pos = 0, speed_in_ms = True):

    # Convert Speed in kph unit
    if speed_in_ms:
        df.Speed = df.Speed * 3.6;
        
    # Speed Mask
    mask_c1 = df.Speed < thres_speed
    
    # Speed Difference Mask
    dspeed = np.roll(df.Speed.values,-10) - df.Speed.values
    mask_c2 = (dspeed >= thres_dspeed)
    
    # RPM Mask
    mask_c3 = df.N > thres_rpm
    
    # Pedal Mask
    mask_c4 = df.PV_AV_CAN > thres_acc_pdl_pos
    
    # Longitudinal Acc Mask
    mask_c5 = df.AccelX > thres_long_acc

    # index_list = df.index[mask_c1 & mask_c2 & mask_c3 & mask_c4 & mask_c5]
    index_list = df.index[mask_c1 & mask_c2 & mask_c3 & mask_c4]
    
    return index_list

In [43]:
get_suddendepature(df)

Int64Index([], dtype='int64')

## 급감속*
---
### 조건
1. Vehicle Speed more than __*10km/h*__ [data.Speed]
2. Deccelerate over __*9km/h*__ per second [data.Speed 변화량]
3. Longitudinal Decceleration over __*1.5m/s2*__ (2번 조건과 충돌됨, C2 : 2.5m/s2) [data.AccelX 음수값]
4. Brake Pedal pressed __*True*__ [data.BrakeLight] -> data.DriverBraking? data.CF_DriBkeStat?

In [44]:
# df.DriverBraking.value_counts()
df.CF_DriBkeStat.value_counts()

0.0    30304
1.0    24647
Name: CF_DriBkeStat, dtype: int64

In [45]:
def get_suddendecel(df, thres_speed = 10, thres_dspeed = -9, thres_long_acc = -1.5,  speed_in_ms = True):

    # Convert Speed in kph unit
    if speed_in_ms:
        df.Speed = df.Speed * 3.6;
        
    # Speed Mask
    mask_c1 = df.Speed > thres_speed
    
    # Speed Difference Mask
    dspeed = np.roll(df.Speed.values,-10) - df.Speed.values
    mask_c2 = (dspeed <= thres_dspeed)
    
    # Longitudinal Acc Mask
    mask_c3 = df.AccelX < thres_long_acc

    # break pedal pressed == True
    # mask_c4 = (df.CF_DriBkeStat == 1)
    mask_c4 = (df.DriverBraking == 1)

    # index_list = df.index[mask_c1 & mask_c2 & mask_c3 & mask_c4]
    index_list = df.index[mask_c1 & mask_c2 & mask_c4]
    
    return index_list
    

In [46]:
get_suddendecel(df)

Int64Index([    2,     3,     4,     5,     6,     7,     8,     9,    10,
               11,
            ...
            54888, 54889, 54890, 54891, 54892, 54893, 54926, 54927, 54950,
            54951],
           dtype='int64', length=6426)

## 급정지*
---
### 조건
1. Deccelerate over __*9km/h*__ per second [data.Speed]
2. Final Speed under __*5km/h*__ [data.Speed 변화량]
3. Longitudinal Decceleration over __*1.5m/s2*__ (1번 조건과 충돌됨, C1 : 2.5m/s2) [data.AccelX 음수값]
4. Brake Pedal pressed more than {__*COUNT*__} when vehicle stopped (2번 조건과 충돌됨) [data.BrakeLight]

In [47]:
def get_suddenstop(df, thres_speed = 5, thres_dspeed = -9, thres_long_acc = -1.5,  speed_in_ms = True):

    # Convert Speed in kph unit
    if speed_in_ms:
        df.Speed = df.Speed * 3.6;
        
    # Speed Mask
    mask_c1 = df.Speed < thres_speed
    
    # Speed Difference Mask
    dspeed = np.roll(df.Speed.values,-10) - df.Speed.values
    mask_c2 = (dspeed <= thres_dspeed)
    
    # Longitudinal Acc Mask
    mask_c3 = df.AccelX < thres_long_acc

    # break pedal pressed == True
    # mask_c4 = (df.CF_DriBkeStat == 1)
    mask_c4 = (df.DriverBraking == 1)

    # index_list = df.index[mask_c1 & mask_c2 & mask_c3 & mask_c4]
    index_list = df.index[mask_c1 & mask_c2 & mask_c4]
    
    return index_list

In [48]:
get_suddenstop(df)

Int64Index([], dtype='int64')

## 급진로변경
---
### 실제 라벨
1. Turnsignal on [data.CF_Gway_TSigLHSw],[data.CF_Gway_TSigRHSw]
---
### 조건 (사용x)
1. Vehicle Speed more than __*30km/h*__
2. Yaw rate over __*10deg*__ per second (0.175rad/s)
3. Accel/Deccel over __*2km/h*__ per second in __*5s*__ window (__*10km/s per 10s__*)
4. **Lateral** Acceleration over __*1.5m/s2*__ (횡방향이 아니라 종방향인가요? 전달받은 조건은 종방향으로 되어 있었는데, 해당 경우 걸리는게 없어서 횡으로 변경)

In [49]:
def get_stepsteer(df, direction):
    ## direction = 'right' / 'left'
    
    if direction == 'left':
        mask_c1 = df.CF_Gway_TSigLHSw == 1
    
    elif direction == 'right':
        mask_c1 = df.CF_Gway_TSigRHSw == 1

    index_list = df.index[mask_c1]

    return index_list

In [50]:
get_stepsteer(df, 'left')

Int64Index([  160,   161,   162,   163,   164,   165,   166,   167,   168,
              169,
            ...
            54235, 54236, 54237, 54238, 54239, 54240, 54241, 54242, 54243,
            54244],
           dtype='int64', length=1852)

In [51]:
get_stepsteer(df, 'right')

Int64Index([ 3137,  3138,  3139,  3140,  3141,  3142,  3143,  3144,  3145,
             3146,
            ...
            54942, 54943, 54944, 54945, 54946, 54947, 54948, 54949, 54950,
            54951],
           dtype='int64', length=1867)

## 급앞지르기
---
### 조건
1. Vehicle Speed more than __*30km/h*__
2. Yaw rate over __*10deg*__ per second (0.175rad/s)
3. Accel/Deccel over __*5km/h*__ per second
4. **Longitudinal** Acceleration over __*1.5m/s2*__
5. Engine RPM exceed __*2500*__

In [52]:
def get_takeover(df, thres_speed = 30, thres_dspeed = 5, thres_long_acc = 1.5, thres_yawrate = 10, thres_rpm =2500, angle_in_degree = False, speed_in_ms = True):
    
    # Convert Speed in kph unit
    if speed_in_ms:
        df.Speed = df.Speed * 3.6;
        
    if angle_in_degree:
        df.GyroZ = np.rad2deg(df.GyroZ)
    # Speed Mask
    mask_c1 = df.Speed > thres_speed
    
    # Yaw Rate Mask
    mask_c2 = df.GyroZ > thres_yawrate
    
    # Speed Difference Mask
    dspeed = np.roll(df.Speed.values,-10) - df.Speed.values
    mask_c3 = (dspeed >= thres_dspeed) | (dspeed <= -thres_dspeed)
    
    # Longitudinal Acc Mask
    mask_c4 = df.AccelX > thres_long_acc
    
    # RPM Mask
    mask_c5 = df.N > thres_rpm
    
    index_list = df.index[mask_c1 & mask_c2 & mask_c3 &  mask_c4 & mask_c5]
    
    return index_list

In [53]:
get_takeover(df)

Int64Index([], dtype='int64')

## 급회전*
---
### 조건
1. Vehicle Speed more than __*20km/h*__ [data.Speed]
2. Cumulative sum of yaw rate in range __*60~120degree*__ [data.GyroZ 적분치의 절대값]
3. Lateral Acceleration over __*1.5m/s2*__ [data.AccelY]

In [54]:
def get_suddenturn(df):
    data_list = df[(df.Speed * 3.6 > 20) * (df.AccelY * 3.6 > 1.5)]
    timestamp = df.TimeStamp
    speed = data_list.Speed * 3.6
    gz = data_list.GyroZ
    ay = data_list.Speed * 3.6
    
    index_list = data_list.index
    
    flat_list = index_list - list(range(len(index_list)))
    unique_list = pd.unique(flat_list)
    event_list = []
    
    for value in unique_list:
        event_index_list = index_list[flat_list == value]
        
        temp_gz = gz[event_index_list]
        
        yaw_rate = np.sum(temp_gz) / len(temp_gz) * (timestamp.iloc[temp_gz.index[-1]] - timestamp.iloc[temp_gz.index[0]])
        
        if 60 <= yaw_rate <= 120:
            event_index_list = event_index_list.astype('int')
            event_index = event_index_list[0]
            event_list += [{"longitude":float(df.Longitude[event_index]), "latitude":float(df.Latitude[event_index]), "timestamp":float(timestamp[event_index]), "timestampList":timestamp[event_index_list].tolist()}]
    
    return event_list

In [55]:
get_suddenturn(df)

[]

## 급유턴
---
### 조건
1. Vehicle Speed more than __*20km/h*__
2. Cumulative sum of yaw rate in range __*60~120degree*__
3. Peak yaw rate over __*30deg/s*__ (0.52rad/s)
4. Lateral Acceleration over __*4m/s2*__

In [56]:
def get_steputurn(df, thres_speed = 20, thres_window = 100, thres_yaw = [160,200], thres_yawrate = 30, thres_lat_acc = 4, angle_in_degree = False, speed_in_ms = True):
    
    # Convert Speed in kph unit
    if speed_in_ms:
        df.Speed = df.Speed * 3.6;
        
    if angle_in_degree:
        df.GyroZ = np.rad2deg(df.GyroZ)
    # Speed Mask
    mask_c1 = df.Speed > thres_speed
    
    # Yaw Mask
    # mask_c2 = (np.convolve(df.GyroZ, np.ones(thres_window, dtype=int),'same') > thres_yaw[0]) & (np.convolve(df.GyroZ, np.ones(thres_window, dtype=int),'same') < thres_yaw[1])
    heading = np.rad2deg(np.unwrap(np.deg2rad(df.Heading)))
    dheading = np.roll(heading, -thres_window) - heading
    mask_c2 = (abs(dheading)>thres_yaw[0]) & (abs(dheading)<thres_yaw[1])
    
    # Peak Yaw Rate
    rollmax = df.GyroZ.abs().rolling(thres_window).max()
    mask_c3 = (rollmax > thres_yawrate) | (rollmax < -thres_yawrate)
    
    # Longitudinal Acc Mask
    mask_c4 = df.AccelY > thres_lat_acc
    
    event_list = []
    index_list = df.index[mask_c1 & mask_c2 & mask_c3 & mask_c4]
    
    flat_index = index_list - list(range(len(index_list)))
    unique_index = pd.unique(flat_index)
    for value in unique_index:
        event_index_list = index_list[flat_index == value].astype('int')
        event_index = [(np.median(event_index_list)).astype('int')]
        if (df.TimeStampUnix.iloc[event_index_list[-1]]-df.TimeStampUnix.iloc[event_index_list[0]]) < 3:
            continue
        event_list += [{"lateralacceleration":float(np.mean(df.AccelY[event_index_list])),"longitude":float(df.Longitude[event_index]), "latitude":float(df.Latitude[event_index]), "timestamp":float(df.TimeStampUnix[event_index]), "timestampList":df.TimeStampUnix[event_index_list].tolist()}]
    return event_list

In [57]:
get_steputurn(df)

[]

In [58]:
get_stepgas_idx = get_stepgas(df)
get_suddendepature_idx = get_suddendepature(df)
get_suddendecel_idx = get_suddendecel(df)
get_suddenstop_idx = get_suddenstop(df)
get_stepsteer_idx_1 = get_stepsteer(df,'left')
get_stepsteer_idx_2 = get_stepsteer(df,'right')
get_takeover_idx = get_takeover(df)
get_suddenturn_idx = get_suddenturn(df)
get_steputurn_idx = get_steputurn(df)

In [59]:
len(df)

54952

In [60]:
cat = np.zeros(len(df))
cat[get_stepgas_idx] = 1
cat[get_suddendepature_idx] = 2
cat[get_suddendecel_idx] = 3
cat[get_suddenstop_idx] = 4
cat[get_takeover_idx] = 7
cat[get_suddenturn_idx] = 8
cat[get_steputurn_idx] = 9
cat[get_stepsteer_idx_1] = 5
cat[get_stepsteer_idx_2] = 6
np.count_nonzero(cat)

10971

In [61]:
df['maneuver'] = cat
df

,Var1,TimeStamp,TQI_TCU_INC,G_SEL_DISP,F_TCU,TCU_TYPE,TCU_OBD,SWI_GS,GEAR_TYPE,TQI_TCU,...,Speed,Heading,positionFix,pDOP,hAcc,timeFix,numSV,Distance,lateral_offset,maneuver
0,0,1.648695e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.036201,0.0
1,1,1.648695e+09,0.0,5.0,0.0,1.0,0.0,1.0,8.0,99.609375,...,5.293720e+07,144.41330,1.0,14.0,9.0,1.0,16.0,0.55860,0.023580,3.0
2,2,1.648695e+09,0.0,5.0,0.0,1.0,0.0,1.0,8.0,99.609375,...,5.296089e+07,145.16798,1.0,14.0,9.0,1.0,16.0,1.67605,0.016444,3.0
3,3,1.648695e+09,0.0,5.0,0.0,1.0,0.0,1.0,8.0,99.609375,...,5.298458e+07,145.51950,1.0,14.0,9.0,1.0,16.0,2.79400,0.013939,3.0
4,4,1.648695e+09,0.0,5.0,0.0,1.0,0.0,1.0,8.0,99.609375,...,5.288034e+07,145.33434,1.0,13.0,10.0,1.0,16.0,3.91110,0.015211,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54947,54947,1.648701e+09,0.0,5.0,0.0,1.0,0.0,0.0,8.0,99.609375,...,1.112430e+08,0.01079,1.0,16.0,6.0,1.0,16.0,54636.58049,-4.791561,6.0
54948,54948,1.648701e+09,0.0,5.0,0.0,1.0,0.0,0.0,8.0,99.609375,...,1.115794e+08,0.16694,1.0,16.0,6.0,1.0,16.0,54638.93458,0.000000,6.0
54949,54949,1.648701e+09,0.0,5.0,0.0,1.0,0.0,0.0,8.0,99.609375,...,1.112145e+08,0.12313,1.0,16.0,6.0,1.0,16.0,54641.28245,0.000000,6.0
54950,54950,1.648701e+09,0.0,5.0,0.0,1.0,0.0,0.0,8.0,99.609375,...,1.110061e+08,0.11534,1.0,16.0,6.0,1.0,16.0,54643.62559,0.000000,6.0


In [62]:
df.to_csv(file_path + 'data_10Hz_labeled.csv')